## Regression test on MyModel -> GPES

In [33]:
import torch
import signatory
import numpy as np
from scipy import optimize

def phi(x, C, a):
  '''
  x: signature norm squared
  C: phi function parameter, it controls how strict the normalization is
  a: phi function parameter, tipically fixed to 1 in order to avoid too much hyperparameter
  '''
  if x > C:
    return C + (C**(1+a))*(C**(-a) - x**(-a))/a
  else:
    return x
  

def dilatation(x, C, a, M, d):
  '''
  x: an array with dimension batch x signature
  C, a : phi function parameters
  M : truncation level (called L in the paper)
  d : dimension of the time series we are computing the signature of (!! if time augmentation is deployed, then the dimension is increased by 1)
  '''
  xNumpy = x.detach().numpy()

  coefficients = np.zeros((xNumpy.shape[0], (M+1)))
  normalizz = np.zeros((xNumpy.shape[0], 1))

  for i in range(0,xNumpy.shape[0]):
      normQuad = 1 + np.sum(xNumpy[i]**2) #signature norm squared
      coefficients[i, 0] = 1-phi(normQuad,C,a)
      for j in range(1, (M+1)):
         coefficients[i, j] = np.sum(xNumpy[i, int(((d**j-1)/(d-1)-1)):int(((d**(j+1)-1)/(d-1)-1))]**2)
      def polin(input):
        xMonomials = np.zeros((M+1)) + 1
        for k in range(1, (M+1)):
            xMonomials[k] = input**(2*k)
        return np.dot(xMonomials, coefficients[i])
      normalizz[i] = optimize.brentq(polin, 0, 2)
      if normalizz[i] > 1:
        normalizz[i] = 1
  return torch.from_numpy(normalizz)


''' grad computation of dilatation function (corollary B.14 in the paper) '''
class Normalization(torch.autograd.Function):
    @staticmethod
    def forward(self, input, C, M, d, exponents): #a=1, input should have dimension batch x length_sign, M is the truncation level, d the dimension of the timeseries, C normalization constant, exponents is useful to define correctly the dilatation
      '''
      input: an array batch x signature, it is x in the previous function
      C, M, d: as in the previous function
      exponents: a vector as long as the signature, it has d times 1, d^2 times 2,..., d^M times the value M
      '''
      norm = dilatation(input, C, 1, M, d)
      self.save_for_backward(input, exponents)
      self.C = C
      self.M = M
      self.d = d
      self.norm = norm # batch x 1
      return norm.to(torch.float32)

    @staticmethod
    def backward(self, grad_output):
      '''
      grad_output: grad of upper layers
      '''
      input,exponents = self.saved_tensors
      normToSquarePower = (self.norm**(2*exponents)).to(torch.float64) #batch x length_sign
      normToSquarePowerMinus1 = (self.norm**(2*exponents-1)).to(torch.float64) #batch x length_sign
      denominator = torch.sum((input**2)*(normToSquarePowerMinus1),1,keepdim=True) #batch x 1
      inputnorm = (1+torch.sum(input**2, 1, keepdim=True))**(1/2) # batch x 1 ,norm of the pre-normalized signature

      # computing phi derivative based on the 2 branches of the phi function
      phiDerivative = torch.zeros(input.shape[0], 1, dtype=torch.float64) #batch x 1
      
      # second branch
      index1 = torch.where(inputnorm[:,0]>(self.C)**(1/2))[0]
      if len(index1) > 0:
        phiDerivative[index1,:] = 2*(self.C)**2/(inputnorm[index1,:]**3)
      
      # first branch
      index2=torch.where(inputnorm[:,0]<=(self.C)**(1/2))[0]
      if len(index2) > 0:
        phiDerivative[index2,:] = 2*inputnorm[index2,:]
      
      # Numerator
      Numerator = normToSquarePower - (phiDerivative/(2*inputnorm))*torch.ones(input.shape[0], int((self.d**(self.M+1)-1)/(self.d-1)-1)) # batch x length_sign
      Numerator = input*Numerator

      gradient = Numerator/denominator # batch x length_sign
      return grad_output*gradient, None, None, None, None
  
''' Triangular: transformed a vector into a lower triangular matrix '''
class Triangular(torch.nn.Module):
  def __init__(self, dim, L2, x_indices, y_indices):
    '''
    dim: dimension of the starting time series
    L2: number of new time instants
    x_indices, y_indices: explained in the model construction (below)
    '''
    super(Triangular, self).__init__()
    self.dim = dim
    self.L2 = L2
    self.x_indices = x_indices
    self.y_indices = y_indices


  def forward(self,x): # x is of size batch x int((int(L2*(L2+1)/2)-int((L2-alp)*(L2-alp+1)/2))*(int(dim*(dim+1)/2)))
    A = torch.zeros((x.shape[0], self.L2*self.dim, self.L2*self.dim))
    A[:,torch.Tensor.long(self.x_indices), torch.Tensor.long(self.y_indices)] = x
    return A


''' Preparation with time augmentation: combines original time series and values sampled, then applies time augmentation '''
class PreparationWithTimeAugmentation(torch.nn.Module):
  def __init__(self, order, timesteps_cut, dim, extended_order):
    '''
    dim: as in the previous function
    timesteps_cut: number of time steps, sum of known and new time steps
    order, extended_order: explained in the model
    '''
    super(PreparationWithTimeAugmentation, self).__init__()
    self.order = order
    self.extended_order = extended_order
    self.cut = timesteps_cut
    self.d = dim

  def forward(self, x, y):
    '''    
    x: starting time series
    y: new values sampled
    '''
    timesteps = x[:, :self.cut] # time instants: before known and then new ones
    values = x[:, self.cut:] # starting time series
    values = torch.cat((values, y), 1) # concatenate values with the new values
    
    # reorder values
    values = values[:, self.extended_order.type(torch.LongTensor)]
    values = values.reshape([values.shape[0], self.cut, self.d])
    
    # adding time component
    timesteps = timesteps[:, self.order]
    path = torch.cat((values, timesteps.unsqueeze(2)), 2)
    return path


class MyModel(torch.nn.Module):
  def __init__(self, L1, L2, dim, order, extended_order, alpha, level, number_classes, C, a, K):
    '''
    L1: number of known time instants, i.e. length of the time series
    L2: number of new time instants
    order: in Preparationwithtimeaugmentation we concatenate the starting values and the sampled one, order reorganize them (FOR EXAMPLE L1=100, L2=99, THEN ORDER=[0,100,1,101,2,102,..] IF THE NEW TIME INSTANTS ARE THE MIDDLE POINTS)
    extended_order: as order but takes into account the dimension of the time series (FOR EXAMPLE D=2, L1=100,L2=99 SO STARTING TIME SERIES HAS 200 VALUES AND NEW VALUES ARE 198, SO EXTENDED ORDER=[0,1,200,201,2,3,202,203,...])
    ALP: how many subdiagonals of the lower triangular matrix are non zero, alpha=L2 means no zero in the lower part
    level: signature truncation level
    number_classes: number of labels in the classification problem
    C, a: phi function parameters
    K: numbers of augmented paths generated
    dim: dimension of the starting time series
    '''
    super(MyModel, self).__init__()
    self.K = K
    self.C = C
    self.a = a
    self.alpha = alpha
    self.L1 = L1
    self.L2 = L2
    self.dim = dim
    self.order = order
    self.extended_order = extended_order

    # compute how much elements in the lower triangular matrix
    MatrixEl = int((int(L2*(L2+1)/2) - int((L2-alpha)*(L2-alpha+1)/2)) * (int(dim*(dim+1)/2)))
    self.level = level
    self.number_classes = number_classes

    # number of components of the signature (remember we have time augmentation)
    self.outputSigDim = int(((dim+1)**(level+1)-1)/(dim)-1)
    # bulding the exponents useful for normalization procedure
    self.exponents = torch.ones(int(((dim+1)**(level+1)-1)/dim-1)).to(torch.float64)
    for j in range(2, (level+1)):
        self.exponents[int((((dim+1)**j-1)/(dim)-1)):int((((dim+1)**(j+1)-1)/(dim)-1))] = torch.ones((int(dim+1)**j))*j

    #needed to reshape first layer output into a matrix (in triangular function)
    tril_indices = torch.tril_indices(row=(dim), col=(dim), offset=0)
    x_indices = torch.zeros(int(L2*(L2+1)/2) - int((L2-alpha)*(L2-alpha+1)/2), dtype=torch.int32)
    for i in range(alpha):
      x_indices[i*L2-int(i*(i-1)/2):(i+1)*L2-int((i+1)*i/2)] = torch.arange(i, L2, 1)

    y_indices=torch.zeros(int(L2*(L2+1)/2) - int((L2-alpha)*(L2-alpha+1)/2), dtype=torch.int32)
    for i in range(alpha):
      y_indices[i*L2-int(i*(i-1)/2):(i+1)*L2-int((i+1)*i/2)] = torch.arange(0, L2-i, 1)

    self.x_indicesFull = torch.zeros(int((int(L2*(L2+1)/2) - int((L2-alpha)*(L2-alpha+1)/2))*(int(dim*(dim+1)/2))), dtype=torch.int32)
    self.y_indicesFull = torch.zeros(int((int(L2*(L2+1)/2) - int((L2-alpha)*(L2-alpha+1)/2))*(int(dim*(dim+1)/2))), dtype=torch.int32)
    for j in range(x_indices.shape[0]):
      self.x_indicesFull[(j*int(dim*(dim+1)/2)):((j+1)*int(dim*(dim+1)/2))] = (x_indices[j]*dim) + tril_indices[0]
      self.y_indicesFull[(j*int(dim*(dim+1)/2)):((j+1)*int(dim*(dim+1)/2))] = (y_indices[j]*dim) + tril_indices[1]

    self.MeanLayer = torch.nn.Linear((L1*(dim+1)+L2), (L2*dim)) # (L1 + L2 + L1*d, L2*d)
    self.SqrtCovLayer = torch.nn.Linear((L1*(dim+1)+L2), MatrixEl)
    self.FinalLayer = torch.nn.Linear(self.outputSigDim, self.number_classes)
    self.GaussianSampler = torch.distributions.Normal(0, 1)
    self.SignatureLayer = signatory.Signature(self.level)
    self.LogSoftmax = torch.nn.LogSoftmax(1)

  def forward(self, x):
    mean = self.MeanLayer(x)
    sqrtCov = self.SqrtCovLayer(x)
    sqrtCovMatrix = Triangular(self.dim, self.L2, self.x_indicesFull, self.y_indicesFull)(sqrtCov)
    epsilon = self.GaussianSampler.sample(torch.Size([x.shape[0], self.L2 * self.dim, self.K]))
    newValues = torch.bmm(sqrtCovMatrix, epsilon) + mean.unsqueeze(2)

    signatures = torch.zeros(x.shape[0], self.outputSigDim, self.K)
    for i in range(self.K):
      path = PreparationWithTimeAugmentation(self.order, self.L1 + self.L2, self.dim, self.extended_order)(x, newValues[:, :, i])
      sig = self.SignatureLayer(path)
      norm = Normalization.apply(sig.to(torch.float64), self.C, self.level, self.dim + 1, self.exponents)
      signatures[:, :, i] = (sig * (norm**self.exponents)).type(torch.float32)

    self.MeanSig = torch.mean(signatures, 2)
    output = self.FinalLayer(self.MeanSig)
    output = self.LogSoftmax(output)
    return output

In [34]:
from lib.model import GPES

In [35]:
begin, end, number, division, dim = 0, 1, 100, 1, 1
'''
begin = first time steps
end = last known time steps
division = 1 means we are taking the middle points as new time instants -> L2 = L1 - 1
number = L1
dim = 1 - one dimensional
'''

# generating the time steps
known_times = torch.linspace(begin, end, number)
new_times = torch.zeros(division*(number-1))
for i in range(0,(number-1)):
  new_times[(division*i):(division*(i+1))] = torch.linspace(known_times[i], known_times[i+1], (division+2))[1:(1 + division)]

# Length of known values and new values
L1 = known_times.shape[0]
L2 = new_times.shape[0]

timesteps = torch.cat((known_times, new_times),axis=0)
timesteps_sorted, order = torch.sort(timesteps)

extended_order = torch.zeros(dim*order.size(0))
for i in range(order.size(0)):
  extended_order[(i*dim):((i+1)*dim)] = torch.arange(order[i]*dim, (order[i] + 1)*dim)

In [36]:
batch_size = 60
x = torch.rand(batch_size, L1 + L2 + L1)
y = (torch.rand(batch_size) > 0.5).long()

alpha = 5
level = 3
number_classes = 2
C = 5e3
a = 1
K = 10

In [37]:
modelOld = MyModel(
    L1=L1, 
    L2=L2, 
    dim=dim, 
    order=order, 
    extended_order=extended_order, 
    alpha=alpha, 
    level=level, 
    number_classes=number_classes, 
    C=C, 
    a=a, 
    K=K
)

In [38]:
model = GPES(
    L1=L1, 
    L2=L2, 
    dim=dim, 
    order=order, 
    extended_order=extended_order, 
    alpha=alpha, 
    level=level, 
    number_classes=number_classes, 
    C=C, 
    a=a, 
    K=K,
)

In [39]:
# same initialization

model.load_state_dict(modelOld.state_dict())

<All keys matched successfully>

In [40]:
# forward pass reg-test

torch.random.manual_seed(0)
yhatOld = modelOld(x)
torch.random.manual_seed(0)
yhat = model(x)

assert torch.allclose(yhatOld, yhat)

# backward pass reg-test

import torch.nn as nn
criterion = nn.CrossEntropyLoss()

lossOld = criterion(yhatOld, y)  # Compute loss
lossOld.backward()  # Backward pass

loss = criterion(yhat, y)  # Compute loss
loss.backward()  # Backward pass

with torch.no_grad():
    for pOld, p in zip(modelOld.parameters(), model.parameters()):
        assert torch.allclose(pOld, p)
        assert torch.allclose(pOld.grad, p.grad)

## Submitting GPU experiments on RDS

In [1]:
import sys
import os
sys.path.append('/rds/general/user/ll1917/home/esig/gp-esig-classifier') # to add when running on remote Jupyter server
os.chdir('/rds/general/user/ll1917/home/esig/gp-esig-classifier') # to add when running on remote Jupyter server

In [10]:
!python train.py -use_cuda -dataset Bidim -martingale_indices 0 1

/bin/bash: which: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `which'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
/bin/bash: scl: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `scl'
/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
100%|█████████████████████████████████████████████| 3/3 [00:10<00:00,  3.66s/it]
Epoch 1/1000
Training Loss: 1.8082, Training Accuracy: 32.67%.
Validation Loss: 1.6407, Validation Accuracy: 32.53%
Improved! Saving model...
100%|█████████████████████████████████████████████| 3/3 [00:10<00:00,  3.65s/it]
Epoch 2/1000
Training Loss: 1.5611, Training Accuracy: 38.53%.
^C
Traceback (most recent call last):
  File "train.py", line 284, in <module>
    train(args)
  File "train.py", line 200, in train
    outputs = model(